In [54]:
'''
YOUR TASKS:

1. SPECIFY THE URL OR PATH TO YOUR DATA, REMEMBER - YOU FIRST HAVE TO DOWNLOAD IT.
2. CLICK RUN ALL OR RESTART AND RUN ALL BUTTON TO RUN THE CODE. THIS CODE RUNS IN A CHRONOLOGICAL MANNER - THE TOP BLOCKS MUST PRECEED THE BOTTOM ONES,
                                                                                                            i.e. don't run stuff from the bottom first, otherwise the code won't work properly.
3. NEAR THE END OF THE CODE, YOU WILL SEE YOUR EPOCH METRICS, AS I STATED IN CLASS, YOU WANT THE ACCURACY TO APPROACH 100% =1.0 AND THE LOSS TO APPROACH 0
4. IN CASE IF YOUR MODEL IS STRUGGLING TO REACH THESE RESULTS AND IS INSTEAD STUCK AT A LOW ACCURACY, NAVIGATE TO THE BLOCK OF CODE WITH HYPERPARAMETERS AND CHANGE THESE NUMBERS,
THEN, TRY RUNNING THE CODE AGAIN AND SEE HOW YOUR MODEL IS PERFORMING; REPEAT UNTIL YOU REACH A PLAUSIBLE RESULT (ONE THAT IS BETTER THAN A COIN TOSS IN TERMS OF PREDICTION ACCURACY IN THE WORST CASE)
5. SAVE YOUR RESULTS => THE CODE INCLUDES A BLOCK WHICH WILL SAVE YOUR MODEL TO A .h5 FILE,
 - KEEP THE OUTPUTS OF THIS NOTEBOOK AS WELL, YOU WILL WANT TO DEMONSTRATE HOW THE TRAINING PROCESS WENT.
 - BE MINDFUL OF THE TIME YOU HAVE IF YOU CHOOSE TO USE GOOGLE COLLABORATORY INSTEAD OF YOUR OWN MACHINE AS THEIR VMs ONLY OFFER YOU A COUPLE OF HOURS OF USAGE.
'''

"\nYOUR TASKS:\n\n1. SPECIFY THE URL OR PATH TO YOUR DATA, REMEMBER - YOU FIRST HAVE TO DOWNLOAD IT.\n2. CLICK RUN ALL OR RESTART AND RUN ALL BUTTON TO RUN THE CODE. THIS CODE RUNS IN A CHRONOLOGICAL MANNER - THE TOP BLOCKS MUST PRECEED THE BOTTOM ONES, \n                                                                                                            i.e. don't run stuff from the bottom first, otherwise the code won't work properly.\n3. NEAR THE END OF THE CODE, YOU WILL SEE YOUR EPOCH METRICS, AS I STATED IN CLASS, YOU WANT THE ACCURACY TO APPROACH 100% =1.0 AND THE LOSS TO APPROACH 0\n4. IN CASE IF YOUR MODEL IS STRUGGLING TO REACH THESE RESULTS AND IS INSTEAD STUCK AT A LOW ACCURACY, NAVIGATE TO THE BLOCK OF CODE WITH HYPERPARAMETERS AND CHANGE THESE NUMBERS,\nTHEN, TRY RUNNING THE CODE AGAIN AND SEE HOW YOUR MODEL IS PERFORMING; REPEAT UNTIL YOU REACH A PLAUSIBLE RESULT (ONE THAT IS BETTER THAN A COIN TOSS IN TERMS OF PREDICTION ACCURACY IN THE WORST CASE)\n5. SAVE YOUR 

In [55]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import time

In [56]:
# Function to read the data
def read_data(filename):
    data = pd.read_csv(filename)

    # Extract features
    features = data.iloc[:,1 :-1].values

    # Extract and encode labels
    label_encoder = LabelEncoder()
    data.iloc[:, -1] = label_encoder.fit_transform(data.iloc[:, -1])
    labels = data.iloc[:, -1].values

    return features, labels
# Read the data
data_url = 'https://raw.githubusercontent.com/camilleb2700/ENGIN-442/main/user_13.csv'# enter your data URL or PATH here in enclosed in " " IF YOUR NOTEBOOK IS IN THE SAME FOLDER AS data.csv, which is your data you downloaded, then this will run as it is.
                    # otherwise, you have to specify a different path or url
x, y = read_data(data_url)

<ipython-input-56-bab6b6b75e96>:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.iloc[:, -1] = label_encoder.fit_transform(data.iloc[:, -1])


In [57]:
df = pd.read_csv(data_url) ## THIS AND THE BLOCK OF CODE IMMEDIATELY AFTER THIS ARE OPTIONAL, THIS IS INTENDED TO HELP YOU VISUALIZE YOUR DATASET

In [58]:
df # AS STATED ABOVE, YOU MAY REMOVE THIS AND THE ABOVE BLOCKS OF CODE IF YOU LIKE, THIS WILL HAVE NO

,Time (Seconds),x-rotation,y-rotation,z-rotation,x-position,y-position,z-position,label
0,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,Sit_VidCall_Phone
1,0.008333,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,Sit_VidCall_Phone
2,0.016667,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,Sit_VidCall_Phone
3,0.025000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,Sit_VidCall_Phone
4,0.033333,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,Sit_VidCall_Phone
...,...,...,...,...,...,...,...,...
81092,31.425000,58.627224,-16.453003,100.837242,0.963660,1.294864,1.035364,Walk_WebBrowse_Tablet
81093,31.433333,58.508793,-16.198683,100.496178,0.960310,1.294741,1.032513,Walk_WebBrowse_Tablet
81094,31.441667,58.372505,-15.928641,100.154335,0.956939,1.294636,1.029681,Walk_WebBrowse_Tablet
81095,31.450000,58.301109,-15.527400,99.811371,0.953483,1.294595,1.026789,Walk_WebBrowse_Tablet


In [59]:
x.shape ### IN THE FOLLOWING PARTS OF THE CODE, MAKE SURE THAT EVERYTING IS LOADED CORRECTLY, YOU SHOULD HAVE A DATASET WITH 22 CLASSES, (n-rows, 6 columns, later another dimension will be added here)

(81097, 6)

In [60]:
y.shape

(81097,)

In [61]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21])

In [62]:
x[0]

array([-0., -0., -0., -0.,  0.,  0.])

In [63]:
y[0] # these were sample rows from the data for your to inspect, you should see a 6 by 1 vector above and a single value here

0

In [64]:
# Split the data into training, validation, and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [65]:
# Reshape the data
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [66]:
# Determine the input shape and number of classes
input_shape = (x_train.shape[1], 1)
num_classes = len(np.unique(y))

In [67]:
num_classes # must have 22

22

In [68]:
input_shape

(6, 1)

In [69]:
x_train.shape

(58389, 6, 1)

In [70]:
y_train.shape

(58389,)

In [71]:
x_train[0]

array([[72.823593],
       [-1.559806],
       [18.949818],
       [ 0.245982],
       [ 1.402932],
       [ 0.185015]])

In [72]:
y_train[0]

20

In [73]:
# Transformer Encoder function with dropout and L2 regularization
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1, l2_reg=0.01):

    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed-forward network with dropout and L2 regularization
    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(res)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    '''
    YOU MAY WANT TO ADD OR REMOVE THESE TWO LINES HERE PRECISELY IN THIS PART OF THE CODE, PAY ATTENTION TO THE INDENTATION, PYTHON REQUIRES CONSISTENCY IN ORDER FOR THE CODE TO RUN CORRECTLY
    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    '''

    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(inputs.shape[-1], activation = 'softmax')(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    return x + res

In [74]:
# Transformer Decoder function
def transformer_decoder(inputs, enc_outputs, head_size, num_heads, ff_dim, dropout=0.1, l2_reg=0.01):
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(res, enc_outputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = x + res

    # Feed-forward network
    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    '''
    YOU MAY WANT TO ADD OR REMOVE THESE TWO LINES HERE PRECISELY IN THIS PART OF THE CODE, PAY ATTENTION TO THE INDENTATION, PYTHON REQUIRES CONSISTENCY IN ORDER FOR THE CODE TO RUN CORRECTLY
    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    '''

    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(inputs.shape[-1], activation = 'softmax')(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    return x + res


In [75]:
# Build the model
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0, l2_reg=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs

    # Encoder
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout, l2_reg)


    # Decoder
    decoder_input = keras.Input(shape=input_shape)
    dec = decoder_input
    for _ in range(num_transformer_blocks):
        dec = transformer_decoder(dec, x, head_size, num_heads, ff_dim, dropout, l2_reg)

    x = layers.Flatten()(x)
    dec = layers.Flatten()(dec)
    x = layers.Concatenate()([x, dec])

    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model([inputs, decoder_input], outputs)
    return model

In [76]:
# Define the hyperparameters
"""
# THIS IS THE PART YOU SHOULD PLAY AROUND WITH
# YOU MAY REDUCE THESE NUMBERS IF YOUR MACHINE IS OTHERWISE INCAPABLE OF RUNNING THE CODE"""

head_size = 4
num_heads = 4
ff_dim = 256
num_transformer_blocks = 4
mlp_units = [256]
dropout = 0.1
mlp_dropout = 0.1
initial_lr = 0.001
l2_reg=0.01

In [77]:
# Build the model
model = build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout, mlp_dropout)

In [78]:
# Create the optimizer with the initial learning rate
optimizer = Adam(learning_rate=initial_lr)

In [79]:
# Compile the model
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [80]:
# Create the ReduceLROnPlateau callback
lr_callback = ReduceLROnPlateau(factor=0.1, patience=50)

In [81]:
# Record the start time
start_time = time.time()

In [82]:
model.fit([x_train, x_train], y_train,
          batch_size=64,
          epochs=100,
          validation_data=([x_val, x_val], y_val),
          callbacks=[lr_callback])

Epoch 1/100
913/913 [==============================] - 104s 73ms/step - loss: 2.0220 - accuracy: 0.6026 - val_loss: 0.6405 - val_accuracy: 0.7773 - lr: 0.0010
Epoch 2/100
913/913 [==============================] - 62s 68ms/step - loss: 0.6379 - accuracy: 0.7827 - val_loss: 0.4829 - val_accuracy: 0.8217 - lr: 0.0010
Epoch 3/100
913/913 [==============================] - 62s 68ms/step - loss: 0.5116 - accuracy: 0.8220 - val_loss: 0.3913 - val_accuracy: 0.8656 - lr: 0.0010
Epoch 4/100
913/913 [==============================] - 63s 69ms/step - loss: 0.4439 - accuracy: 0.8461 - val_loss: 0.3333 - val_accuracy: 0.8845 - lr: 0.0010
Epoch 5/100
913/913 [==============================] - 60s 66ms/step - loss: 0.3960 - accuracy: 0.8634 - val_loss: 0.3621 - val_accuracy: 0.8619 - lr: 0.0010
Epoch 6/100
913/913 [==============================] - 62s 68ms/step - loss: 0.3567 - accuracy: 0.8773 - val_loss: 0.2983 - val_accuracy: 0.8991 - lr: 0.0010
Epoch 7/100
913/913 [==============================

In [83]:
# Record the end time
end_time = time.time()

In [84]:
# Calculate the total running time
running_time = end_time - start_time
print("Total Running Time: {:.2f} seconds".format(running_time))

Total Running Time: 5971.30 seconds


In [85]:
# Evaluate the model
loss, accuracy = model.evaluate([x_test, x_test], y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

254/254 [==============================] - 6s 22ms/step - loss: 0.0645 - accuracy: 0.9793
Test loss: 0.06445014476776123, Test accuracy: 0.9792848229408264


In [86]:
# Save the trained model
model.save("my_model_otto.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
